# Benchmarking [GPT-4o-mini](https://openai.com/index/introducing-o3-and-o4-mini/)

## Libraries

In [9]:
from openai import OpenAI
import random
from time import sleep 
from pathlib import Path
import numpy as np
import json
import os
import pandas as pd
from tqdm.rich import tqdm
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    precision_score, recall_score, f1_score, accuracy_score
)
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.text import Text
import warnings
from concurrent.futures import ThreadPoolExecutor, as_completed

tqdm.pandas(desc="Fetching moderation scores")
console = Console()
warnings.filterwarnings("ignore")

## Global variables

In [10]:
ROOT = Path("../..")
DATA_DIR = ROOT / "data"
BENCHMARK_PATH = DATA_DIR / "benchmark_jigsaw" / "benchmark_jigsaw.csv"
output_path = DATA_DIR / "benchmark_jigsaw" / "eng-gpt-4o-mini_simple_prompt.csv"
API_KEY_PATH = DATA_DIR / "confidential" / "GPT_API.txt"
console = Console()
model = "gpt-4o-mini"
system_prompt = (Path(".") / "API_SYSTEM_PROMPT_simple.txt").read_text().strip()
prompt = (Path(".") / "API_PROMPT_simple.txt").read_text().strip()

In [11]:
os.environ["HTTP_PROXY"] = "socks5h://127.0.0.1:1080"
os.environ["HTTPS_PROXY"] = "socks5h://127.0.0.1:1080"

In [12]:
client = OpenAI(api_key=API_KEY_PATH.read_text().strip())

## Load dataset

In [13]:
df = pd.read_csv(BENCHMARK_PATH, encoding="utf-8")
df = df.dropna(subset=["content", "label"])
df["label"] = df["label"].astype(int)

label_counts = df["label"].value_counts().reset_index()
label_counts.columns = ["label", "count"]
table = Table(title="Label Counts", show_lines=True)
table.add_column("Label", justify="center", style="cyan")
table.add_column("Count", justify="center", style="yellow")
table.add_row("Total Rows", str(len(df)))
for _, row in label_counts.iterrows():
    table.add_row(str(row["label"]), str(row["count"]))
console.print(table, justify="left")
df.sample(5, random_state=42)

     Label Counts                                                                                                  
┏━━━━━━━━━━━━┳━━━━━━━┓                                                                                             
┃   Label    ┃ Count ┃                                                                                             
┡━━━━━━━━━━━━╇━━━━━━━┩                                                                                             
│ Total Rows │  396  │                                                                                             
├────────────┼───────┤                                                                                             
│     0      │  198  │                                                                                             
├────────────┼───────┤                                                                                             
│     1      │  198  │                                                                                             
└────────────┴───────┘

,msg_id,original_text,content,label
78,ddb11abeca87bb3c,birth place \n\n YC was not born in Jonesboro...,lieu de naissance \n\nYC n'est pas né à Jones...,0
277,6e9826f166b7d7fe,Nazis like you deserve worse than death. \n\nI...,Des nazis comme toi méritent pire que la mort....,1
250,5bff63ef016af9be,You can rationalize your dishonesty however yo...,Tu peux rationaliser ton malhonnêteté comme tu...,1
55,e8ea81d9eac81735,Deletion\nThis has been tagged for Speedy Dele...,Suppression \nCeci a été marqué pour une supp...,0
391,bf5aae5b17fb0b77,because I am a huge faggot,parce que je suis un énorme pédé,1


## Define prediction function

In [14]:
def predict(text, max_retries=5, delay_base=0.5):
    input_text = prompt + f"« {text} »" + "\n Is this message toxic ?\n"
    for attempt in range(max_retries):
        sleep_time = delay_base + delay_base * (2 ** attempt)  # Exponential backoff
        sleep(sleep_time)
        try:
            response = client.responses.create(
                model=model,
                input=[
                    {
                        "role": "developer",
                        "content": system_prompt,
                    },
                    {
                        "role": "user",
                        "content": input_text,
                    }
                ]
            )
            result = response.output_text.strip().lower()
            if result == "toxic":
                return 1
            elif result == "non-toxic":
                return 0
            else:
                console.print(f"[red]Unexpected response: '{result}'[/red]")
                return None
        except Exception as e:
            console.print(f"[red]Error: {e}[/red]")
    console.print(f"[red]Max retries reached for: {text}[/red]")
    return None

def safe_predict(text):
    try:
        return predict(text)
    except Exception as e:
        console.print(f"[red]Error for text: {text} → {e}[/red]")
        return None

In [15]:
# List of texts to classify
texts = df["original_text"].tolist()

# before launching:
results = [None] * len(texts)

with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_idx = {
        executor.submit(safe_predict, txt): idx
        for idx, txt in enumerate(texts)
    }
    for future in tqdm(as_completed(future_to_idx), total=len(texts)):
        idx = future_to_idx[future]
        results[idx] = future.result()

Output()

In [16]:
df["toxicity_score"] = results

In [17]:
df['prediction'] = results

In [18]:
for i, row in df.sample(5, random_state=42).iterrows():
    content = Text(row['content'], style="bold")
    toxicity = f"[yellow]Toxicity Score:[/yellow] [bold]{int(row['toxicity_score'])}[/bold]"
    label = f"[cyan]Label:[/cyan] [bold]{row['label']}[/bold]"
    panel = Panel.fit(
        f"{content}\n\n{toxicity}\n{label}",
        title=f"Exemple {i+1}",
        border_style="magenta"
    )
    console.print(panel)

╭─────────────── Exemple 79 ───────────────╮
│ lieu de naissance                        │
│                                          │
│ YC n'est pas né à Jonesboro, en Géorgie. │
│                                          │
│ Toxicity Score: 0                        │
│ Label: 0                                 │
╰──────────────────────────────────────────╯

╭────────────────── Exemple 278 ──────────────────╮
│ Des nazis comme toi méritent pire que la mort.  │
│                                                 │
│ Je vais m'assurer que tu souffres horriblement. │
│                                                 │
│ Toxicity Score: 1                               │
│ Label: 1                                        │
╰─────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Exemple 251 ──────────────────────────────────────────────────╮
│ Tu peux rationaliser ton malhonnêteté comme tu veux, mais même Wikipédia a un article sur la mosquée de Ground  │
│ Zero, espèce de connard.                                                                                        │
│                                                                                                                 │
│ Toxicity Score: 1                                                                                               │
│ Label: 1                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Exemple 56 ─────────────────────────────────────────────╮
│ Suppression                                                                                         │
│ Ceci a été marqué pour une suppression rapide car c'est le même article précédemment supprimé sous. │
│                                                                                                     │
│ Toxicity Score: 0                                                                                   │
│ Label: 0                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────── Exemple 392 ───────────╮
│ parce que je suis un énorme pédé │
│                                  │
│ Toxicity Score: 1                │
│ Label: 1                         │
╰──────────────────────────────────╯

## Metrics & Report        

| Metric                     | Formula                                           | Interpretation                                                                                                       |
| -------------------------- | ------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- |
| **Precision**              | `TP / (TP + FP)`                                  | Of the samples predicted **toxic**, how many were **actually toxic**? <br>→ High precision = **low false positives** |
| **Recall** *(Sensitivity)* | `TP / (TP + FN)`                                  | Of the **actual toxic** samples, how many did we **correctly identify**? <br>→ High recall = **low false negatives** |
| **F1-score**               | `2 * (Precision * Recall) / (Precision + Recall)` | Harmonic mean of precision and recall. <br>→ Best when **balance** is needed                                         |
| **Accuracy**               | `(TP + TN) / (TP + TN + FP + FN)`                 | Fraction of all correct predictions (toxic and non-toxic). <br>→ Can be misleading on imbalanced data                |
| **ROC AUC**                | Area under the ROC Curve                          | Measures the **ranking ability** of the classifier. <br>→ Higher = better separation of toxic vs. non-toxic          |


In [19]:
y_true = df["label"]
y_pred = df["toxicity_score"].astype(int)

In [20]:
# Rapport de classification
report = classification_report(y_true, y_pred, digits=3, output_dict=True)
table = Table(title="Classification Report", show_lines=True)
table.add_column("Classe", style="cyan", justify="center")
table.add_column("Precision", justify="center")
table.add_column("Recall", justify="center")
table.add_column("F1-score", justify="center")
table.add_column("Support", justify="center")

for label, metrics in report.items():
    if label in ["accuracy", "macro avg", "weighted avg"]:
        continue
    table.add_row(
        str(label),
        f"{metrics['precision']:.3f}",
        f"{metrics['recall']:.3f}",
        f"{metrics['f1-score']:.3f}",
        f"{int(metrics['support'])}"
    )
# Ajout de l'accuracy
table.add_row(
    "[bold yellow]Accuracy[/bold yellow]",
    "-",
    "-",
    f"{report['accuracy']:.3f}",
    "-"
)
console.print(table)

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)
cm_table = Table(title="Matrice de Confusion", show_lines=True)
cm_table.add_column(" ", style="bold")
cm_table.add_column("Prédit 0", justify="center")
cm_table.add_column("Prédit 1", justify="center")
cm_table.add_row("Réel 0", str(cm[0, 0]), str(cm[0, 1]))
cm_table.add_row("Réel 1", str(cm[1, 0]), str(cm[1, 1]))
console.print(cm_table)

# ROC AUC
try:
    auc = roc_auc_score(y_true, df["toxicity_score"])
    console.print(Panel(f"[bold green]ROC AUC Score : {auc:.3f}[/bold green]", title="ROC AUC"))
except:
    console.print(Panel("[bold red]Impossible de calculer le ROC AUC.[/bold red]", title="ROC AUC"))

                Classification Report                 
┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┓
┃  Classe  ┃ Precision ┃ Recall ┃ F1-score ┃ Support ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━┩
│    0     │   0.939   │ 0.929  │  0.934   │   198   │
├──────────┼───────────┼────────┼──────────┼─────────┤
│    1     │   0.930   │ 0.939  │  0.935   │   198   │
├──────────┼───────────┼────────┼──────────┼─────────┤
│ Accuracy │     -     │   -    │  0.934   │    -    │
└──────────┴───────────┴────────┴──────────┴─────────┘

      Matrice de Confusion      
┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃        ┃ Prédit 0 ┃ Prédit 1 ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ Réel 0 │   184    │    14    │
├────────┼──────────┼──────────┤
│ Réel 1 │    12    │   186    │
└────────┴──────────┴──────────┘

╭──────────────────────────────────────────────────── ROC AUC ────────────────────────────────────────────────────╮
│ ROC AUC Score : 0.934                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Save the predictions

In [21]:
df['toxicity_score'] = df['toxicity_score'].astype(int)
df

,msg_id,original_text,content,label,toxicity_score,prediction
0,e6865c831da40459,Combining 'Official requests for clemency' sec...,Combiner la section 'Demandes officielles de c...,0,0,0
1,cff2b456e5059b47,Web \n\nIs this her own website? 113.22.84.65,Web\n\nEst-ce son propre site web ? 113.22.84.65,0,0,0
2,880d3d608f37bee3,"RFCU \n\nFYI, I have opened Wikipedia:Requests...","RFCU \n\nPour info, j'ai ouvert Wikipedia:Requ...",0,0,0
3,dd29d52a4320a2e4,Monitor for cleanup: \nAs of the date signed b...,Surveillance pour nettoyage : \nÀ la date sig...,0,0,0
4,d70e773c975839e1,I concur and have done so.,Je suis d'accord et je l'ai fait.,0,0,0
...,...,...,...,...,...,...
391,bf5aae5b17fb0b77,because I am a huge faggot,parce que je suis un énorme pédé,1,1,1
392,8947f2c12aea0a32,"If no-one beats me to it, I'll knock something...","Si personne ne me devance, je vais bricoler qu...",1,0,0
393,ccf05c5d4b791836,"ur a faggot, fuck off and get a real job and s...","T'es un pédé, casse-toi et trouve un vrai boul...",1,1,1
394,ae7a53526025c0e2,"I SAID SHUT UP, YOU STUPID, STUPID...","J'AI DIT FERME TA GUEULE, ESPÈCE DE BÊTE, BÊTE...",1,1,1


In [22]:
output_path

PosixPath('../../data/benchmark_jigsaw/eng-gpt-4o-mini_simple_prompt.csv')

In [23]:
df.to_csv(output_path, index=False, encoding="utf-8")